In [ ]:
# default_exp photon_data
%load_ext autoreload
%autoreload 2

# Photon Data

> Load timed photon data

Remarks on loading the photon data around a source, subject to a GTI.

In [ ]:
#hide

from nbdev.showdoc import *

In [ ]:
#export
import os, sys
import healpy
import pickle
import pandas as pd
import numpy as np
import pyarrow.parquet as pq
from light_curves.config import MJD, UTC
from light_curves.load_gti import get_gti

def get_photon_data(config: 'configuration data',
#                     files:  'file locations',
                    source: 'Source data',
                    gti:   'Good Time Interval'=None,
                    use_cache=True,
                    nest=True):
    """
    Read photon data from a Parquet dataset, select cone around the
    source, use exposure to add exposures, return DataFrame
    """
    
    def summarize(df):
        emin,emax = config.energy_range or (config.energy_edges[0],config.energy_edges[-1])
        print(f'\n\tSelected {len(df)} photons within {config.radius}'\
              f' deg of  ({source.l:.2f},{source.b:.2f})')
        print(f'\tEnergies: {emin:.1f}-{emax:.0f} MeV')
        ta,tb = df.iloc[0].time, df.iloc[-1].time
        print(f'\tDates:    {UTC(ta):16} - {UTC(tb)}'
            f'\n\tMJD  :    {ta:<16.1f} - {tb:<16.1f}')
        
    key = f'photons_{source.name}'
    files = config.files
    photon_data = config.cache.get(key)
    if photon_data is not None:
        if config.verbose>1: 
            print(f'restored {key} from cache')
            summarize(photon_data)
        return photon_data
    
    # check GTI
    if gti is None:
        gti = get_gti(config, files.gti)

    # cone geometry stuff: get corresponding pixels and center vector
    l,b,radius = source.l, source.b, config.radius
    cart = lambda l,b: healpy.dir2vec(l,b, lonlat=True)
    conepix = healpy.query_disc(config.nside, cart(l,b), np.radians(radius), nest=nest)
    center = healpy.dir2vec(l,b, lonlat=True)

    ebins = config.energy_edges
    ecenters = np.sqrt(ebins[:-1]*ebins[1:]);
    band_limits = 2*np.searchsorted(ecenters, config.energy_range) if config.energy_range is not None else None

    def load_photon_data(table, tstart):
        """For a given month table, select photons in cone, add tstart to times,
        return DataFrame with band, time, pixel, radius
        """
        allpix = np.array(table.column('nest_index'))

        def cone_select(allpix, conepix, shift=None):
            """Fast cone selection using NEST and shift
            """
            if shift is None:
                return np.isin(allpix, conepix)
            assert nest, 'Expect pixels to use NEST indexing'
            a = np.right_shift(allpix, shift)
            c = np.unique(np.right_shift(conepix, shift))
            return np.isin(a,c)

        # a selection of all those in an outer cone
        incone = cone_select(allpix, conepix, 13)

        # times: convert to double, add to start, convert to MJD
        time = MJD(np.array(table['time'],float)[incone]+tstart)
        in_gti = gti(time)
        if np.sum(in_gti)==0:
            print(f'WARNING: no photons in GTI for month {month}!', file=sys.stderr)

        pixincone = allpix[incone][in_gti]

        # distance from center for all accepted photons
        ll,bb = healpy.pix2ang(config.nside, pixincone,  nest=nest, lonlat=True)
        cart = lambda l,b: healpy.dir2vec(l,b, lonlat=True)
        t2 = np.degrees(np.array(np.sqrt((1.-np.dot(center, cart(ll,bb)))*2), np.float32))

        # assemble the DataFrame, remove those outside the radius
        out_df = pd.DataFrame(np.rec.fromarrays(
            [np.array(table['band'])[incone][in_gti], time[in_gti], pixincone, t2],
            names='band time pixel radius'.split()))

        # apply final selection for radius and energy range

        if band_limits is None: return out_df.query(f'radius<{radius}')

        return out_df.query(f'radius<{radius} & {band_limits[0]} < band < {band_limits[1]}')

    # get the monthly-partitioned dataset and tstart values
    dataset = files.data/'dataset'
    tstart_dict= pickle.load(open(files.data/'tstart.pkl', 'rb'))
    months = tstart_dict.keys()

    if config.verbose>0:
        print(f'Loading  {len(months)} months from Arrow dataset {dataset}\n', end='')

    dflist=[]
    for month, tstart in tstart_dict.items(): #months:
        table= pq.read_table(dataset, filters=[f'month == {month}'.split()])
        #tstart = tstart_dict[month]
        d = load_photon_data(table, tstart)
        if d is not None:
            dflist.append(d)
            if config.verbose>1: print('.', end='')
        else:
            if config.verbose>1: print('x', end='')
            continue

    assert len(dflist)>0, '\nNo photon data found?'
    df = pd.concat(dflist, ignore_index=True)
    if config.verbose>0:
        summarize(df)
         

    config.cache.add(key, df)
    if config.verbose>1:
        print(f'added {key} to cache')
    
    return df

In [ ]:
from light_curves.config import Config,  PointSource

config = Config()

if config.valid:
    source = PointSource('Geminga')

    print(f'Test loading a photon data set, for source {source.name}')
    photon_data = get_photon_data(config,  source )
else:
    print('Not testing since no files.')

Test loading a photon data set, for source Geminga
restored photons_Geminga from cache

	Selected 1313726 photons within 5 deg of  (195.13,4.27)
	Energies: 100.0-1000000 MeV
	Dates:    2008-08-04 15:46 - 2019-08-03 01:17
	MJD  :    54682.7          - 58698.1         


In [ ]:
photon_data.iloc[0]

band      6.000000e+00
time      5.468266e+04
pixel     6.738278e+06
radius    6.983809e-01
Name: 0, dtype: float64

In [ ]:
config.cache.path

Path('/tmp/lc_cache')

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()
!date

Converted 00_config.ipynb.
Converted 01_effective_area.ipynb.
Converted 02_load_gti.ipynb.
Converted 03_exposure.ipynb.
Converted 04_photon_data.ipynb.
Converted 05_weights.ipynb.
Converted 07_cells.ipynb.
Converted 09_poisson.ipynb.
Converted 10_loglike.ipynb.
Converted 11_lightcurve.ipynb.
Converted 12_instructions.ipynb.
Converted 13_kerr_comparison.ipynb.
Converted 14_bayesian.ipynb.
Converted index.ipynb.
Tue Dec 15 17:11:25 PST 2020
